In [1]:
getwd()
if ( is.null(environment()$this_notebook_dir) ) {
    this_notebook_dir <- getwd()
    setwd(paste0(getwd(), '/..'))
}
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/NA_fix"

[1] "/mnt/hdd/develop/kaggle/house-prices/R"

In [20]:
source('helpers.R')

kaggle.house.loadLibraries()
data <- kaggle.house.loadData()

df.LotFrontage <- data$combined %>% select(-dataSource) %>% filter(!is.na(LotFrontage))

df.LotFrontage.vs.numeric <- df.LotFrontage %>% select_if(is.numeric)

df.LotFrontage.vs.char <- df.LotFrontage %>% 
    select(starts_with('LotFrontage'),  
           which(sapply(df.LotFrontage, is.character)) %>% names)

df.LotFrontage %>% dim
df.LotFrontage.vs.numeric %>% dim
df.LotFrontage.vs.char %>% dim

[1] 2433   79

[1] 2433   35

[1] 2433   45

In [21]:
df.LotFrontage %>% select(-LotFrontage, -Utilities) %>% 
    map(function (col) {  
            lm(LotFrontage ~ attr, tibble(attr=col, LotFrontage=df.LotFrontage$LotFrontage))
    }) -> models.lst

tibble(
    var = names(models.lst),
    mod = models.lst,
    glance = map(mod, broom::glance)
) -> models.df

models.df %>% head(2) %>% print

models.df %>% select(-mod) %>% unnest(glance) %>% arrange(desc(r.squared)) %>% head

# A tibble: 2 x 3
         var      mod                glance
       <chr>   <list>                <list>
1 MSSubClass <S3: lm> <data.frame [1 x 11]>
2   MSZoning <S3: lm> <data.frame [1 x 11]>


var,r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
MSSubClass,0.3539728,0.3502324,18.81790,94.6340420,3.433373e-217,15,-10585.14332,21202.2866,21295.0367,856246.0,2418
BldgType,0.2787318,0.2775436,19.84257,234.5732511,1.820384e-170,5,-10719.16444,21450.3289,21485.1102,955970.6,2428
Neighborhood,0.2676187,0.2603192,20.07771,36.6626965,1.468147e-143,25,-10737.76508,21527.5302,21678.2491,970700.0,2408
LotArea,0.2399977,0.2396850,20.35583,767.6744036,4.455289e-147,2,-10782.80015,21571.6003,21588.9909,1007309.0,2431
X1stFlrSF,0.2099907,0.2096658,20.75379,646.1791231,1.315681e-126,2,-10829.90688,21665.8138,21683.2044,1047080.2,2431
PoolQC,0.1712089,-0.1050548,82.69119,0.6197301,5.692922e-01,3,-50.68187,109.3637,110.1526,41027.0,6


In [32]:
df.LotFrontage.vs.char %>%
select(-Utilities) %>%
gather(attr.name, attr.val, -LotFrontage) %>%
group_by(attr.name) %>% 
nest %>% 
mutate(
    mod = map(data, ~lm(LotFrontage ~ attr.val, .)),
    glance = map(mod, broom::glance)
) %>% 
select(-data, -mod) %>% 
unnest(glance) %>%
arrange(desc(r.squared)) %>%
head

attr.name,r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
MSSubClass,0.3539728,0.3502324,18.81790,94.6340420,3.433373e-217,15,-10585.14332,21202.2866,21295.0367,856246.03,2418
BldgType,0.2787318,0.2775436,19.84257,234.5732511,1.820384e-170,5,-10719.16444,21450.3289,21485.1102,955970.63,2428
Neighborhood,0.2676187,0.2603192,20.07771,36.6626965,1.468147e-143,25,-10737.76508,21527.5302,21678.2491,970700.03,2408
PoolQC,0.1712089,-0.1050548,82.69119,0.6197301,5.692922e-01,3,-50.68187,109.3637,110.1526,41027.00,6
Alley,0.1587058,0.1541085,16.21692,34.5219924,1.953858e-08,2,-776.91834,1559.8367,1569.4977,48126.87,183
MSZoning,0.1374468,0.1360241,21.67323,96.6052348,2.316218e-76,5,-10920.38758,21852.7752,21887.5490,1139092.89,2425
